## 피마 인디언 당뇨병 예측

In [1]:
import numpy as np
import pandas as pd

- 데이터 전처리 및 탐색

In [2]:
df = pd.read_csv('pima-indians-diabetes.csv', skiprows=9, header=None)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pima-indians-diabetes.csv'

In [ ]:
# 1. Number of times pregnant
# 2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
# 3. Diastolic blood pressure (mm Hg)
# 4. Triceps skin fold thickness (mm)
# 5. 2-Hour serum insulin (mu U/ml)
# 6. Body mass index (weight in kg/(height in m)^2)
# 7. Diabetes pedigree function
# 8. Age (years)
# 9. Class variable (0 or 1)
df.columns = ['P', 'G', 'BP', 'S', 'I', 'BMI', 'D', 'Age', 'Target']
df.head()

,P,G,BP,S,I,BMI,D,Age,Target
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# 결측치 확인
df.isna().sum().sum()

0

In [ ]:
# X, y 데이터를 ndarray로 추출
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X.shape, y.shape

((768, 8), (768,))

In [ ]:
# y 값의 분포
np.unique(y, return_counts=True)

(array([0, 1], dtype=int64), array([500, 268], dtype=int64))

- Train/Test 데이터셋 분리

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2023
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

- GridSearchCV - Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dtc = DecisionTreeClassifier(random_state=2023)
params = {
    'max_depth': [2, 5, 8],
    'min_samples_split': [2, 3, 4]
}
grid_dt = GridSearchCV(dtc, params, scoring='accuracy', cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2023),
             param_grid={'max_depth': [2, 5, 8],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [ ]:
grid_dt.best_params_

{'max_depth': 5, 'min_samples_split': 2}

In [ ]:
dtc = DecisionTreeClassifier(random_state=2023)
params = {
    'max_depth': [4, 5, 6],
    'min_samples_split': [2, 3, 4]
}
grid_dt = GridSearchCV(dtc, params, scoring='accuracy', cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2023),
             param_grid={'max_depth': [4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [ ]:
grid_dt.best_params_

{'max_depth': 6, 'min_samples_split': 3}

- best estimator로 예측 및 평가

In [ ]:
best_dt = grid_dt.best_estimator_
pred = best_dt.predict(X_test)
rf = pd.DataFrame({'y 실제값': y_test, 'y 예측값': pred})
rf.head()

,y 실제값,y 예측값
0,0,0
1,1,1
2,0,0
3,1,1
4,1,1


In [ ]:
best_dt.score(X_test, y_test)

0.7207792207792207

- 특정 데이터에 대해 적용

In [ ]:
X_test[10], y_test[10]

(array([  0.   , 101.   ,  62.   ,   0.   ,   0.   ,  21.9  ,   0.336,
         25.   ]),
 0)

In [ ]:
test_data, test_target = X_test[10], y_test[10]

In [ ]:
result = best_dt.predict(test_data.reshape(1, -1))[0]
result

0